In [ ]:
from collections import namedtuple
from pyspark.sql.functions import col,split, explode
inputDataset="5c89949bef707515181cf97a" # 2019 Summit Platform Lab Backup postValues
outputDataset="5c927f59012c9615168ba7ec" # Recommendations Input Dataset

In [ ]:
pd1 = spark.read.format("com.adobe.platform.dataset").option('orgId', "DF6233FB5C538CAE0A495EF3@AdobeOrg").load(inputDataset)
pd1.describe()

In [ ]:
#Filtering empty values
item_id = "_experience.analytics.customDimensions.eVars.eVar17"
interactionType = "_experience.analytics.customDimensions.eVars.eVar18"
user_id = "_experience.analytics.customDimensions.eVars.eVar16"
timestamp = "timestamp"

pd1 = pd1.filter(pd1[user_id].isNotNull()).filter(pd1[item_id].isNotNull())

In [ ]:
#Splitting item_id to individual records

pd0 = pd1.withColumn("itemId",explode(split(col(item_id), "\|\|")))
pd0.describe()

In [ ]:
df = pd0.selectExpr("_experience.analytics.customDimensions.eVars.eVar16 as userId",
           "itemId as itemId",
           "_experience.analytics.customDimensions.eVars.eVar18 as interactionType",
           "timestamp as timestamp")

_platformlab05 = namedtuple("_platformlab05", ["userId", "itemId", "interactionType", "timestamp"])
output = df.rdd.map(lambda parts: { "_platformlab05" : _platformlab05(parts[0], parts[1],
                                                                                       parts[2], parts[3]),"timestamp":parts[3]
                           } )
df_output = output.toDF()
df_output.printSchema()

In [ ]:
userToken = spark.sparkContext.getConf().get('spark.yarn.appMasterEnv.USER_TOKEN')
serviceToken = spark.sparkContext.getConf().get('spark.yarn.appMasterEnv.SERVICE_TOKEN')
serviceApiKey = spark.sparkContext.getConf().get('spark.yarn.appMasterEnv.SERVICE_API_KEY')
df_output.write.format("com.adobe.platform.dataset")\
.option('orgId', "DF6233FB5C538CAE0A495EF3@AdobeOrg")\
.option('userToken', userToken)\
.option('serviceToken', serviceToken)\
.option('serviceApiKey', serviceApiKey)\
.save(outputDataset)